In [150]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/284709b96cd84f60930aad7d2ca8bb10
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [25]                : (0.3502214252948761, 0.658504843711853)
COMET INFO:     batch_accuracy [1200]        : (0.125, 0.8125)
COMET INFO:     batch_loss [1200]            : (0.5997779369354248, 2.7833456993103027)
COMET INFO:     epoch_duration [25]          : (5.0, 10.797000000005937)
COMET INFO:     loss [25]                    : (0.8623648881912231, 1.9000506401062012)
COMET INFO:     val_accuracy [25]            : (0.3466666638851166, 0.5355555415153503)
COMET INFO:     val_loss [25]                : (1.3721890449523926, 1.8192678689956665)
COMET INFO:     validate_batch_accuracy [50] : (0.281

In [151]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

In [152]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline
sns.set_style('whitegrid')

In [153]:
# tensorflow 2.7 
import tensorflow as tf 
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,BatchNormalization,Concatenate
from tensorflow.keras.models import Model, Sequential

In [154]:
# scale the numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [155]:
df_train_features = pd.read_csv('data/train_features.csv')
df_train_labels = pd.read_csv('data/train_labels.csv')

df_valid_features = pd.read_csv('data/valid_features.csv')
df_valid_labels = pd.read_csv('data/valid_labels.csv')

df_test_features = pd.read_csv('data/test_features.csv')

In [156]:
df_train_features.head(1)

,trackID,title,tags,loudness,tempo,time_signature,key,mode,duration,vect_1,...,vect_139,vect_140,vect_141,vect_142,vect_143,vect_144,vect_145,vect_146,vect_147,vect_148
0,8424,The Calamity [Explicit],"the, you, to, a, not, in, of, that, are, we, w...",-2.824,131.969,4,0,1,298.13506,52.050795,...,0.001097,0.001161,0.001225,0.001251,0.001238,0.001202,0.00113,0.00104,0.104525,0.335818


In [157]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

num_train_meta_data = df_train_features.iloc[:,3:5]
num_valid_meta_data = df_valid_features.iloc[:,3:5]

num_train_meta_data['duration'] = df_train_features['duration']
num_valid_meta_data['duration'] = df_valid_features['duration']

In [158]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)

num_train_meta = scaler.fit_transform(num_train_meta_data)
num_valid_meta = scaler.fit_transform(num_valid_meta_data)

In [159]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

In [160]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [161]:
x_train = df_train_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_train = df_train_labels['genre'].to_numpy()

x_test = df_valid_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
y_test = df_valid_labels['genre'].to_numpy()

In [162]:
# remove stopwords
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [163]:
x_train = x_train.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
x_test = x_test.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))

In [164]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [165]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df_train_labels['genre']),y=df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

In [166]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [167]:
import ktrain
from ktrain import text

In [168]:
trn2, val2, preproc2 = text.texts_from_array(
    x_train=x_train, y_train=y_train,
    x_test=x_test, y_test=y_test,
    ngram_range=1, 
    maxlen=150,
    preprocess_mode='standard'
)

language: en
Word Counts: 4776
Nrows: 7678
7678 train sequences
train sequence lengths:
	mean : 52
	95percentile : 95
	99percentile : 146
x_train shape: (7678,150)
y_train shape: (7678, 8)
Is Multi-Label? False
450 test sequences
test sequence lengths:
	mean : 50
	95percentile : 101
	99percentile : 129
x_test shape: (450,150)
y_test shape: (450, 8)
task: text classification


In [169]:
model2 = text.text_classifier('fasttext', train_data=trn2, preproc=preproc2, metrics=['accuracy'])

Is Multi-Label? False
compiling word ID features...
maxlen is 150
done.


In [170]:
import math
class MyCustomDataset(ktrain.SequenceDataset):
    def __init__(self, x, y, batch_size=32, shuffle=True):
        # error checks
        err = False
        if type(x) == np.ndarray and len(x.shape) != 2: err = True
        elif type(x) == list:
            for d in x:
                if type(d) != np.ndarray or len(d.shape) != 2:
                    err = True
                    break
        else: err = True
        if err:
            raise ValueError('x must be a 2d numpy array or a list of 2d numpy arrays')
        if type(y) != np.ndarray:
            raise ValueError('y must be a numpy array')
        if type(x) == np.ndarray:
            x = [x]
        # set variables

        super().__init__(batch_size=batch_size)
        self.x, self.y = x, y
        self.indices = np.arange(self.x[0].shape[0])
        self.n_inputs = len(x)
        self.shuffle = shuffle

    # required for instances of tf.keras.utils.Sequence
    def __len__(self):
        """Number of batches in the Sequence.
        """
        return math.ceil(self.x[0].shape[0] / self.batch_size)

    # required for instances of tf.keras.utils.Sequence
    def __getitem__(self, idx):
        """Gets batch at position `idx`.
        """
        # the starting and ending indices of the batch
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = []

        for i in range(self.n_inputs):
            batch_x.append(self.x[i][inds])
        batch_y = np.array(self.y[inds])
        return tuple(batch_x) , batch_y

    # required for instances of ktrain.Dataset
    def nsamples(self):
        return self.x[0].shape[0]

    #required for instances of ktrain.Dataset
    def get_y(self):
        return self.y

    def on_epoch_end(self):
        if self.shuffle:  np.random.shuffle(self.indices)

In [171]:
train_data = MyCustomDataset([num_train] + [num_train_meta] + [trn2[0]] , trn2[1])
valid_data = MyCustomDataset([num_valid] + [num_train_meta] + [val2[0]] , val2[1])

In [172]:
class AudioNumMetaClassifier:
    def __init__(self,num_train,num_train_meta,train_data,num_train_y,num_valid,num_valid_meta,valid_data,num_valid_y,class_weight,early_stop,experiment,text_model):
        # audio data
        self.num_train = num_train
        self.num_valid = num_valid

        # meta data
        self.num_train_meta = num_train_meta
        self.num_valid_meta = num_valid_meta

        # all data and ktrain model
        self.train_data = train_data
        self.valid_data = valid_data
        self.text_model = text_model

        # labels
        self.num_train_y = num_train_y
        self.num_valid_y = num_valid_y

        # class weights and early stopping
        self.class_weight = class_weight
        self.early_stop = early_stop

        # experiment
        self.experiment = experiment
    
    def build_model(self):
        # Build the model
        model = Sequential()
        model.add(
            Dense(
                units=self.experiment.get_parameter("audio_layer_1"),
                activation='elu',
                input_shape=(self.num_train.shape[1],)
            )
        )
        model.add(Dropout(0.5))
        model.add(BatchNormalization())

        model.add(
            Dense(
                units=self.experiment.get_parameter("audio_layer_2"),
                activation='elu'))
        model.add(Dropout(0.3))
        model.add(BatchNormalization())

        model_meta = Sequential()
        model_meta.add(
            Dense(
                units=self.experiment.get_parameter("meta_layer_1"),
                activation='elu',
                input_shape=(self.num_train_meta.shape[1],)
            )
        )
        model_meta.add(Dropout(0.5))
        model.add(BatchNormalization())

        model_meta.add(
            Dense(
                units=self.experiment.get_parameter("meta_layer_2"),
                activation='elu'))
        model_meta.add(Dropout(0.3))
        model_meta.add(BatchNormalization())


        merged_out = Concatenate()([model.output,model_meta.output,self.text_model.output])
        merged_out = Dense(
            units=self.experiment.get_parameter("final_layer"),
            activation='elu')(merged_out)
        merged_out = Dense(8,activation='softmax')(merged_out)

        combined_model = Model([model.input] + [model_meta.input] + [self.text_model.input], [merged_out])
        combined_model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return combined_model
    
    def train_model(self):
        # Train the model
        combined_model = self.build_model()
        model = ktrain.get_learner(
            combined_model,
            train_data=self.train_data,
            val_data=self.valid_data,
            batch_size=self.experiment.get_parameter("batch_size")
        )
        model.fit_onecycle(
            lr=0.001,
            callbacks=[self.early_stop],
            epochs=self.experiment.get_parameter("epochs"),
            class_weight=self.class_weight,
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        # score = model.evaluate([self.num_valid,self.num_valid_meta],self.num_valid_y)
        # LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [173]:
classifier = AudioNumMetaClassifier(num_train,num_train_meta,train_data,num_train_y,num_valid,num_valid_meta,valid_data,num_valid_y,class_weight,early_stop,experiment,model2)

In [174]:
classifier.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "audio_layer_1": {"type": "discrete", "values": [450,500,550,600,700,900,950,1000]},
        "audio_layer_2": {"type": "discrete", "values": [128,256,300,400,450,500,550]},
        "meta_layer_1": {"type": "discrete", "values": [450,500,550,600,700,900,1000]},
        "meta_layer_2": {"type": "discrete", "values": [128,256,300,400,450,500,550]},
        "final_layer": {"type": "discrete", "values": [20,32,64,100,128,256]},
        "batch_size": {"type": "discrete", "values": [16,32,40,64]},
        "epochs": {"type": "discrete", "values": [13,14,15,16,20]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=ac9577df43cc4625a3b73696e8632eb2
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 329280, 'endTime': None, 'id': 'ac9577df43cc4625a3b73696e8632eb2', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'audio_layer_1': {'type': 'discrete', 'values': [450, 500, 550, 600, 700, 900, 950, 1000]}, 'audio_layer_2': {'type': 'discrete', 'values': [128, 256, 300, 400, 450, 500, 550]}, 'batch_size': {'type': 'discrete', 'values': [16, 32, 40, 64]}, 'epochs': {'type': 'discrete', 'values': [13, 14, 15, 16, 20]}, 'final_layer': {'type': 'discrete', 'values': [20, 32, 64, 100, 128, 256]}, 'meta_layer_1': {'type': 'discrete', 'values': [450, 500, 550, 600, 700, 900, 1000]}, 'meta_layer_2': {'type': 'discrete', 'values': [128, 256, 300, 400, 450, 500, 550]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAs



begin training using onecycle policy with max lr of 0.001...
Epoch 1/14
192/192 [==============================] - 6s 23ms/step - loss: 1.8766 - accuracy: 0.3583 - val_loss: 1.4829 - val_accuracy: 0.4667
Epoch 2/14
192/192 [==============================] - 4s 21ms/step - loss: 1.5278 - accuracy: 0.4531 - val_loss: 1.6652 - val_accuracy: 0.4200
Epoch 3/14
192/192 [==============================] - 4s 21ms/step - loss: 1.4076 - accuracy: 0.4838 - val_loss: 1.6535 - val_accuracy: 0.4556
Epoch 4/14
192/192 [==============================] - 4s 22ms/step - loss: 1.3272 - accuracy: 0.5151 - val_loss: 1.6281 - val_accuracy: 0.4889
Epoch 5/14
192/192 [==============================] - 4s 21ms/step - loss: 1.2227 - accuracy: 0.5431 - val_loss: 1.6644 - val_accuracy: 0.4489
Epoch 6/14
192/192 [==============================] - 4s 22ms/step - loss: 1.1805 - accuracy: 0.5604 - val_loss: 1.6551 - val_accuracy: 0.4356


begin training using onecycle policy with max lr of 0.001...
Epoch 1/14
192/1

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/81950fc95aee44d39c456c1ff42cab1f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [12]                : (0.3582964241504669, 0.563948929309845)
COMET INFO:     batch_accuracy [240]         : (0.07500000298023224, 0.699999988079071)
COMET INFO:     batch_loss [240]             : (0.7820491790771484, 2.934331178665161)
COMET INFO:     epoch_duration [12]          : (3.9679999999934807, 7.39100000000326)
COMET INFO:     loss [12]                    : (1.1804945468902588, 1.8766275644302368)
COMET INFO:     val_accuracy [12]            : (0.41999998688697815, 0.5355555415153503)
COMET INFO:     val_loss [12]                : (1.3834078311920166, 1.6988356113433838)
COMET INFO:    



begin training using onecycle policy with max lr of 0.001...
Epoch 1/13
240/240 [==============================] - 8s 23ms/step - loss: 1.8918 - accuracy: 0.3591 - val_loss: 1.5483 - val_accuracy: 0.4533
Epoch 2/13
240/240 [==============================] - 5s 21ms/step - loss: 1.5611 - accuracy: 0.4504 - val_loss: 1.6119 - val_accuracy: 0.4711
Epoch 3/13
240/240 [==============================] - 5s 21ms/step - loss: 1.4251 - accuracy: 0.4840 - val_loss: 1.6265 - val_accuracy: 0.4422
Epoch 4/13
240/240 [==============================] - 5s 21ms/step - loss: 1.3350 - accuracy: 0.5121 - val_loss: 1.4969 - val_accuracy: 0.4889
Epoch 5/13
240/240 [==============================] - 5s 20ms/step - loss: 1.2552 - accuracy: 0.5428 - val_loss: 1.5980 - val_accuracy: 0.4644
Epoch 6/13
240/240 [==============================] - 5s 21ms/step - loss: 1.1891 - accuracy: 0.5619 - val_loss: 1.5530 - val_accuracy: 0.4756
Epoch 7/13
240/240 [==============================] - 5s 20ms/step - loss: 1.15

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/b933d4ce741c44b9851c6443c8aeb40d
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [19]                : (0.3590778708457947, 0.6842927932739258)
COMET INFO:     batch_accuracy [456]         : (0.125, 0.75)
COMET INFO:     batch_loss [456]             : (0.6482904553413391, 2.811835289001465)
COMET INFO:     epoch_duration [19]          : (4.514999999999418, 8.031000000002678)
COMET INFO:     loss [19]                    : (0.7984223365783691, 1.8917553424835205)
COMET INFO:     val_accuracy [19]            : (0.41555556654930115, 0.5422222018241882)
COMET INFO:     val_loss [19]                : (1.4091987609863281, 1.7333006858825684)
COMET INFO:     validate_batch_accuracy [

...

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/ec54896543c2421ba9f7270661b6cec1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [23]                : (0.3200052082538605, 0.668533444404602)
COMET INFO:     batch_accuracy [552]         : (0.125, 0.75)
COMET INFO:     batch_loss [552]             : (0.808335542678833, 2.5297203063964844)
COMET INFO:     epoch_duration [23]          : (1.9380000000091968, 3.7810000000026776)
COMET INFO:     loss [23]                    : (0.8657002449035645, 1.9094011783599854)
COMET INFO:     val_accuracy [23]            : (0.41333332657814026, 0.5600000023841858)
COMET INFO:     val_loss [23]                : (1.3912405967712402, 1.6488409042358398)
COMET INFO:     validate_batch_accuracy 



begin training using onecycle policy with max lr of 0.001...
Epoch 1/14
480/480 [==============================] - 6s 10ms/step - loss: 1.8687 - accuracy: 0.3588 - val_loss: 1.6932 - val_accuracy: 0.3733
Epoch 2/14
480/480 [==============================] - 4s 8ms/step - loss: 1.5476 - accuracy: 0.4430 - val_loss: 1.5552 - val_accuracy: 0.4356
Epoch 3/14
480/480 [==============================] - 4s 9ms/step - loss: 1.3915 - accuracy: 0.4858 - val_loss: 1.4765 - val_accuracy: 0.4889
Epoch 4/14
480/480 [==============================] - 4s 8ms/step - loss: 1.2851 - accuracy: 0.5280 - val_loss: 1.4627 - val_accuracy: 0.5178
Epoch 5/14
480/480 [==============================] - 4s 8ms/step - loss: 1.2137 - accuracy: 0.5465 - val_loss: 1.4252 - val_accuracy: 0.5378
Epoch 6/14
480/480 [==============================] - 4s 9ms/step - loss: 1.1638 - accuracy: 0.5647 - val_loss: 1.6265 - val_accuracy: 0.4489
Epoch 7/14
480/480 [==============================] - 4s 9ms/step - loss: 1.1153 - a

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/41bbbb907c054e90b53374340f38e532
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [25]                : (0.3588173985481262, 0.6865069270133972)
COMET INFO:     batch_accuracy [1200]        : (0.0625, 0.875)
COMET INFO:     batch_loss [1200]            : (0.4730539619922638, 2.784681797027588)
COMET INFO:     epoch_duration [25]          : (3.7339999999967404, 6.10899999999674)
COMET INFO:     loss [25]                    : (0.8005383610725403, 1.8686538934707642)
COMET INFO:     val_accuracy [25]            : (0.3733333349227905, 0.5933333039283752)
COMET INFO:     val_loss [25]                : (1.2181159257888794, 1.6931582689285278)
COMET INFO:     validate_batch_accuracy 



begin training using onecycle policy with max lr of 0.001...
Epoch 1/20
240/240 [==============================] - 3s 10ms/step - loss: 1.7918 - accuracy: 0.3832 - val_loss: 1.4246 - val_accuracy: 0.5178
Epoch 2/20
240/240 [==============================] - 2s 10ms/step - loss: 1.4503 - accuracy: 0.4753 - val_loss: 1.5492 - val_accuracy: 0.4867
Epoch 3/20
240/240 [==============================] - 2s 9ms/step - loss: 1.2924 - accuracy: 0.5255 - val_loss: 1.4941 - val_accuracy: 0.4756
Epoch 4/20
240/240 [==============================] - 2s 9ms/step - loss: 1.1624 - accuracy: 0.5742 - val_loss: 1.4789 - val_accuracy: 0.4800
Epoch 5/20
240/240 [==============================] - 2s 9ms/step - loss: 1.0847 - accuracy: 0.5964 - val_loss: 1.4036 - val_accuracy: 0.5022
Epoch 6/20
240/240 [==============================] - 2s 10ms/step - loss: 1.0050 - accuracy: 0.6192 - val_loss: 1.4876 - val_accuracy: 0.4644
Epoch 7/20
240/240 [==============================] - 2s 9ms/step - loss: 0.9462 -

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/e64c2fdef597479ebe88b30d6cc82544
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [24]                : (0.37939566373825073, 0.6980984807014465)
COMET INFO:     batch_accuracy [576]         : (0.15625, 0.78125)
COMET INFO:     batch_loss [576]             : (0.43188461661338806, 2.5712742805480957)
COMET INFO:     epoch_duration [24]          : (2.0, 3.4530000000086147)
COMET INFO:     loss [24]                    : (0.7691603302955627, 1.826533555984497)
COMET INFO:     val_accuracy [24]            : (0.4555555582046509, 0.5577777624130249)
COMET INFO:     val_loss [24]                : (1.3105894327163696, 1.6065045595169067)
COMET INFO:     validate_batch_accuracy [48] : (

In [200]:
# Build the model
model = Sequential()
model.add(
    Dense(
        units=550,
        activation='elu',
        input_shape=(num_train.shape[1],)
    )
)
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(
    Dense(
        units=300,
        activation='elu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model_meta = Sequential()
model_meta.add(
    Dense(
        units=600,
        activation='elu',
        input_shape=(num_train_meta.shape[1],)
    )
)
model_meta.add(Dropout(0.5))
model.add(BatchNormalization())

model_meta.add(
    Dense(
        units=128,
        activation='elu'))
model_meta.add(Dropout(0.3))
model_meta.add(BatchNormalization())


merged_out = Concatenate()([model.output,model_meta.output,model2.output])
merged_out = Dense(
    units=32,
    activation='elu')(merged_out)
merged_out = Dense(8,activation='softmax')(merged_out)

combined_model = Model([model.input] + [model_meta.input] + [model2.input], [merged_out])
combined_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [201]:
learner = ktrain.get_learner(
            combined_model,
            train_data=train_data,
            val_data=valid_data,
            batch_size=64
        )
learner.fit_onecycle(
    lr=0.001,
    callbacks=[early_stop],
    epochs=14,
    class_weight=class_weight,
)



begin training using onecycle policy with max lr of 0.001...
Epoch 1/14
120/120 [==============================] - 2s 12ms/step - loss: 1.8585 - accuracy: 0.3545 - val_loss: 1.5298 - val_accuracy: 0.4356
Epoch 2/14
120/120 [==============================] - 1s 9ms/step - loss: 1.5040 - accuracy: 0.4459 - val_loss: 1.4578 - val_accuracy: 0.4578
Epoch 3/14
120/120 [==============================] - 1s 9ms/step - loss: 1.3634 - accuracy: 0.4996 - val_loss: 1.4273 - val_accuracy: 0.4911
Epoch 4/14
120/120 [==============================] - 1s 9ms/step - loss: 1.2566 - accuracy: 0.5343 - val_loss: 1.4809 - val_accuracy: 0.4756
Epoch 5/14
120/120 [==============================] - 1s 9ms/step - loss: 1.1648 - accuracy: 0.5698 - val_loss: 1.4371 - val_accuracy: 0.4800
Epoch 6/14
120/120 [==============================] - 1s 9ms/step - loss: 1.0945 - accuracy: 0.5944 - val_loss: 1.3483 - val_accuracy: 0.5289
Epoch 7/14
120/120 [==============================] - 1s 9ms/step - loss: 1.0083 - a

In [202]:
num_test_data = df_test_features.iloc[:,9:]
num_test_meta_data = df_test_features.iloc[:,3:5]
num_test_meta_data['duration'] = df_test_features['duration']

num_test = scaler.fit_transform(num_test_data)
num_test_meta = scaler.fit_transform(num_test_meta_data)

df_test_label = pd.read_csv('test_labels.csv')
num_test_y = pd.get_dummies(df_test_label['genre'])

In [203]:
x_test2 = df_test_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
x_test2 = x_test2.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
x_test2 = np.array(x_test2)

In [204]:
x_text_input = preproc2.preprocess_test(x_test2)

428 test sequences
test sequence lengths:
	mean : 54
	95percentile : 109
	99percentile : 150
x_test shape: (428,150)


In [205]:
test_dataset = [num_test] + [num_test_meta] + [x_text_input[0]]

In [206]:
prediction = learner.model.predict(test_dataset)

In [207]:
predict_labels = [ label.argmax() for label in prediction]
df = pd.DataFrame(predict_labels, columns=['genre_pred'])
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
df['genre_pred'] = df['genre_pred'].map(classes_mapping)
df['trackID'] = df_test_features['trackID']

correct_label = pd.read_csv('test_labels.csv')
# merge df and correct_label by trackID
df_merged = pd.merge(df, correct_label, on='trackID')

from sklearn.metrics import classification_report
print(classification_report(df_merged['genre'], df_merged['genre_pred']))

                       precision    recall  f1-score   support

 classic pop and rock       0.31      0.47      0.37        64
dance and electronica       0.37      0.16      0.22        64
                 folk       0.57      0.59      0.58        73
       jazz and blues       0.18      0.11      0.14        36
                metal       0.62      0.55      0.58        42
                  pop       0.67      0.98      0.80        44
                 punk       0.51      0.47      0.49        40
      soul and reggae       0.41      0.43      0.42        65

             accuracy                           0.47       428
            macro avg       0.46      0.47      0.45       428
         weighted avg       0.45      0.47      0.45       428

